SETUP AND INITAL CONFIG

In [2]:
from unsloth import FastModel
import torch

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


c:\Users\Romaiz\Documents\Python_ML\RL\Fine_Tuning\Gemme3-Unsloth\.venv\Lib\site-packages\triton\knobs.py:212: UserWarning: Failed to find cuobjdump.exe
  warnings.warn(f"Failed to find {binary}")
c:\Users\Romaiz\Documents\Python_ML\RL\Fine_Tuning\Gemme3-Unsloth\.venv\Lib\site-packages\triton\knobs.py:212: UserWarning: Failed to find nvdisasm.exe
  warnings.warn(f"Failed to find {binary}")
c:\Users\Romaiz\Documents\Python_ML\RL\Fine_Tuning\Gemme3-Unsloth\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
W1125 16:14:34.769000 31864 Lib\site-packages\torch\distributed\elastic\multiprocessing\redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.


🦥 Unsloth Zoo will now patch everything to make training faster!


In [ ]:
model, tokenizer = FastModel.from_pretrained(
    model_name="google/gemma-3-1b-it",
    max_seq_length=1024,
    load_in_4bit=True,
    load_in_8bit=False,
)

In [ ]:
model = FastModel.get_peft_model(
    model,
    finetune_vision_layers=False,
    finetune_language_layers=True,
    finetune_attention_modules=True,
    finetune_mlp_modules=True,
    r=16,
    lora_alpha=16,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=3047
)

In [ ]:
from datasets import load_dataset
dataset = load_dataset("madrylab/gsm8k-platinum", "main", split="test")

In [ ]:
dataset

In [ ]:
dataset[0]["question"]

In [ ]:
dataset[0]["answer"]


In [ ]:
def extract_hash_answer(text):
    if "####" not in text: return None
    return text.split("####")[1].strip()
extract_hash_answer(dataset[0]["answer"])

In [ ]:
reasoning = "<reasoning>"
answer = "<answer>"

system_prompt = \
f"""You are given a problem.
Think about the problem and provide your working out.
Place it between {reasoning} and {reasoning}.
Then, provide your solution between {answer}{answer}."""


In [ ]:
dataset = dataset.map(lambda x: {
    "prompt" : [
        {"role": "system", "content": system_prompt},
        {"role": "user",   "content": x["question"]},
    ],
    "answer": extract_hash_answer(x["answer"]),
})

In [ ]:
dataset[0]

In [ ]:
import re

match_format = re.compile(
    rf"^[\s]{{0,}}"\
    rf"{reasoning}.+?{reasoning}.*?"\
    rf"{answer}(.+?){answer}"\
    rf"[\s]{{0,}}$",
    flags = re.MULTILINE | re.DOTALL
)

In [ ]:
match_format.search("<reasoning> Here is my working out<reasoning>"\
                    "<answer>42 <answer>")

In [ ]:
def match_format_exactly(completions, **kwargs):
    scores = []
    for output in completions:
        score = 0
        response = output[0]["content"]
        if match_format.search(response) is not None:
            score += 3
            scores.append(score)
        else:
            scores.append(score)
    return scores

In [ ]:
def match_format_partially(completions, **kwargs):
    scores = []
    for output in completions:
        score = 0
        response = output[0]["content"]
        score += 1 if response.count(reasoning) == 2 else -1
        score += 1 if response.count(answer) == 2 else -1
        scores.append(score)
    return scores

In [ ]:
def check_answer_correctness(prompts, completions, answer, **kwargs):
    question = prompts[0][-1]["content"]
    responses = [output[0]["content"] for output in completions]

    extracted_responses = [
        guess.group(1) if (guess := match_format.search(r)) is not None else None for r in responses
    ]

    scores = []
    for guess, true_answer in zip(extracted_responses, answer):
        score = 0
        if guess is None:
            scores.append(0)
            continue
        if guess == true_answer:
            score += 3
        elif guess.strip() == true_answer.strip():
            score += 1.5
        else:
            try:
                ratio = float(guess) / float(true_answer)
                if ratio >= 0.9 and ratio <= 1.1:
                    score += 0.5
                elif ratio >= 0.8 and ratio <= 1.2:
                    score += 0.25
                else:
                    score -= 1
            except:
                score -= 1
        scores.append(score)
    return scores

In [ ]:
match_numbers = re.compile(
    rf"{answer}.*?([\d\d.]{{1,}})",
    flags= re.MULTILINE | re.DOTALL
)

In [ ]:
match_numbers.findall("<answer> The answer is 42.0 <answer>")

In [ ]:
def check_numbers(prompts, completions, answer, **kwargs):
    question = prompts[0][-1]["content"]
    responses = [output[0]["content"] for output in completions]
    
    extracted_responses = [guess.group(1) if (guess := match_numbers.search(r)) is not None else None for r in responses]

    scores = []
    print("*"*5, f"\nAnswer: {answer[0]}", f"\nExtracted: {extracted_responses[0]}")
    for guess, true_answer in zip(extracted_responses, answer):
        if guess is None:
            scores.append(0)
            continue
        try:
            true_answer = float(true_answer)
            guess = float(guess)
            scores.append(1.5 if guess == true_answer else 0.0)
        except:
            scores.append(0.0)
            continue
    return scores


TRAINING 

In [ ]:
from trl import GRPOConfig, GRPOTrainer

In [ ]:
training_args = GRPOConfig(
    learning_rate= 5e-6,
    adam_beta1= 0.9,
    adam_beta2= 0.99,
    weight_decay= 0.1,
    warmup_ratio= 0.1,
    lr_scheduler_type= "cosine",
    optim= "adamw_8bit",
    logging_steps=1,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    num_generations=8,
    max_prompt_length=256,
    max_completion_length=768,
    max_steps=50,
    save_steps=50,
    max_grad_norm=0.1,
    report_to= "wandb",
    output_dir= "outputs"
)

In [ ]:
trainer = GRPOTrainer(
    model= model,
    processing_class= tokenizer,
    reward_funcs= [
        match_format_exactly,
        match_format_partially,
        check_answer_correctness,
        check_numbers
    ],
    args= training_args,
    train_dataset= dataset
)

In [ ]:
trainer.train()

In [ ]:
model.save_pretrained("gemma-3-1b-finetuned")  # Local saving
tokenizer.save_pretrained("gemma-3-1b-finetuned")  # Local saving
model.push_to_hub("Romaiz_Dabeer/gemma-3-1b-finetuned-unsloth") # Online saving
tokenizer.push_to_hub("Romaiz_Dabeer/gemma-3") # Online saving

In [ ]:
model.save_pretrained_merged("gemma-3-1b-finetuned-unsloth", tokenizer)

In [ ]:
model.push_to_hub_merged(
        "Romaiz_Dabeer/gemma-3-finetune-finetuned-unsloth", tokenizer
    )

In [ ]:
model.push_to_hub_gguf(
        "HF_ACCOUNT/gemma-finetune-gguf",
        tokenizer,
        quantization_method = "Q8_0", # Only Q8_0, BF16, F16 supported
        token = "hf_...",
    )

In [3]:
torch.cuda.empty_cache()